In [1]:
from gernerate_data import load_clas_seg_data
import tensorflow as tf
from sklearn.metrics import classification_report, auc, roc_curve
from keras.utils.np_utils import *
from keras.callbacks import LearningRateScheduler
from models.MLT_net import MTL_classic, MTL_3, MTL3_interaction, MTL_3_2,create_pair_model
# from models.shareLayer import create_pair_model
from sklearn.preprocessing import LabelBinarizer, label_binarize
from utils.losses import dice_coef_loss, dice_coef, dice_p_bce, dice_p_focal, tversky_loss, focal_loss, focal_tversky, \
    p_r_f1_iou, generalized_dice_coeff, generalized_dice_loss
from keras.losses import categorical_crossentropy, mean_squared_error, binary_crossentropy
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, CSVLogger
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import keras.backend as K
import utils_paths
import numpy as np
import pickle
import os

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [2]:
INIT_LR = 2e-4
EPOCHS =200
batch_size = 8
depth = 3
img_size = 224
Name = "MTL_Classic_LE"
GPU = True
target = (img_size, img_size)

if GPU:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "2"


    
print("------------------------------------------------ Preparing model ------------------------------------------------")
model = MTL_classic(img_size, img_size, depth, nClasses=2)
# model = MTL_3(img_size, img_size, depth, nClasses=2)
# model = MTL_3_2(img_size, img_size, depth, nClasses=2)
model.summary()

------------------------------------------------ Preparing model ------------------------------------------------
Instructions for updating:
Colocations handled automatically by placer.


2022-03-25 02:35:47.555590: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-03-25 02:35:47.585295: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200095000 Hz
2022-03-25 02:35:47.588841: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x6ae07e0 executing computations on platform Host. Devices:
2022-03-25 02:35:47.588970: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2022-03-25 02:35:48.196681: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x6b5d080 executing computations on platform CUDA. Devices:
2022-03-25 02:35:48.196731: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2022-03-25 02:35:48.197582: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found device 0 with properties: 
name: Tes

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]          

In [3]:
# trainX_dir = 'dataset/LE/test/images/'
trainX_dir = 'dataset/LE/train/images/'
valX_dir = trainX_dir.replace('train', 'val')
testX_dir = trainX_dir.replace('train', 'test')


train_x,  train_c_y, train_s_y = load_clas_seg_data(trainX_dir, target)
val_x, val_c_y, val_s_y = load_clas_seg_data(valX_dir, target)
test_x, test_c_y, test_s_y = load_clas_seg_data(testX_dir, target)


lb = LabelBinarizer()
train_c_y = lb.fit_transform(train_c_y)
val_c_y = lb.fit_transform(val_c_y)
test_c_y = lb.fit_transform(test_c_y)
train_c_y = to_categorical(train_c_y, 2)
val_c_y = to_categorical(val_c_y, 2)
test_c_y = to_categorical(test_c_y, 2)

dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (32).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (32).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (2)_3f841cfb-612f-45a0-81bd-6fa37e2501f7.png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (2)_3f841cfb-612f-45a0-81bd-6fa37e2501f7.png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (20)_e176a4e9-f580-4c77-b3ed-110bb0c0b818.png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (20)_e176a4e9-f580-4c77-b3ed-110bb0c0b818.png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (394)_7a3a8ee6-fb9c-492a-8458-16264bd81556.png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (394)_7a3a8ee6-fb9c-492a-8458-16264bd81556.png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (393).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (393).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (194).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks

dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (158).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (158).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (260).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (260).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (183)_5a2cb258-4abe-4aad-b9f4-0f3fdb9b182e.png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (183)_5a2cb258-4abe-4aad-b9f4-0f3fdb9b182e.png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (175).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (175).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (207).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (207).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (95).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (95).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (105).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign 

dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (363).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (363).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (227).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (227).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (87).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (87).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (266).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (266).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (129).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (129).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (289).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (289).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (76).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (76).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (195)_14

dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (186).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (186).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (431)_d7c3d11e-b352-4e23-bcda-b534b3588de0.png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (431)_d7c3d11e-b352-4e23-bcda-b534b3588de0.png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (70).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (70).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (189).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (189).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (307).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (307).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (5).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (5).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (27).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (27).

dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (355)_da484afe-bda8-461d-8ade-2f375f3209e0.png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (355)_da484afe-bda8-461d-8ade-2f375f3209e0.png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (162).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (162).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (255)_2a785671-7051-478b-82e0-0bb785d7d1b0.png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (255)_2a785671-7051-478b-82e0-0bb785d7d1b0.png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (264)_2fe0ec91-9b88-4f95-a844-51a6b0a5f9ce.png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (264)_2fe0ec91-9b88-4f95-a844-51a6b0a5f9ce.png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (46).png
dataset/Dataset_BUSI_AN/train/masks/bengin_masks/benign (46).png
dataset/Dataset_BUSI_AN/train/images/bengin_images/benign (335).png
dataset/Dataset_BUSI_AN/train/masks/bengin

...

dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (102).png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (209).png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (209).png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (151)_1440aec6-9669-4d5f-ade0-5b45ddb93272.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (151)_1440aec6-9669-4d5f-ade0-5b45ddb93272.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (16)_48fc6c00-361b-4ad2-9ec4-330d55251543.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (16)_48fc6c00-361b-4ad2-9ec4-330d55251543.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (92)_bf6d3021-89f7-4ce8-9c0f-36b606fa9f14.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (92)_bf6d3021-89f7-4ce8-9c0f-36b606fa9f14.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (156)_efe45797-4162-4cb1-ba41-3dd925758f31.png
dataset/Dataset

dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (41)_9b3d44d5-8327-4624-a375-837228733527.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (63)_e52f8e0e-8d73-4164-9cc6-59419196dc5d.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (63)_e52f8e0e-8d73-4164-9cc6-59419196dc5d.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (173).png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (173).png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (16)_1aa3c254-b407-4f51-9d97-16d5e64c0b6a.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (16)_1aa3c254-b407-4f51-9d97-16d5e64c0b6a.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (30).png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (30).png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (207).png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (207).png
dataset/Dataset_BUSI

dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (82)_7a627f6b-0eca-430e-9c05-e2feeaf491df.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (152).png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (152).png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (110)_0ff37a1b-0799-4871-9785-e692298efe81.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (110)_0ff37a1b-0799-4871-9785-e692298efe81.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (94)_171436fc-3e44-4809-9c81-0bc54a38f3a5.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (94)_171436fc-3e44-4809-9c81-0bc54a38f3a5.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (109)_c18aaa53-c46e-4489-9b32-a6caebde137e.png
dataset/Dataset_BUSI_AN/train/masks/malignant_masks/malignant (109)_c18aaa53-c46e-4489-9b32-a6caebde137e.png
dataset/Dataset_BUSI_AN/train/images/malignant_images/malignant (93)_a2a3d118-05be-4745-

dataset/Dataset_BUSI_AN/val/images/bengin_images/benign (138).png
dataset/Dataset_BUSI_AN/val/masks/bengin_masks/benign (138).png
dataset/Dataset_BUSI_AN/val/images/bengin_images/benign (226).png
dataset/Dataset_BUSI_AN/val/masks/bengin_masks/benign (226).png
dataset/Dataset_BUSI_AN/val/images/bengin_images/benign (373).png
dataset/Dataset_BUSI_AN/val/masks/bengin_masks/benign (373).png
dataset/Dataset_BUSI_AN/val/images/bengin_images/benign (63).png
dataset/Dataset_BUSI_AN/val/masks/bengin_masks/benign (63).png
dataset/Dataset_BUSI_AN/val/images/bengin_images/benign (191).png
dataset/Dataset_BUSI_AN/val/masks/bengin_masks/benign (191).png
dataset/Dataset_BUSI_AN/val/images/bengin_images/benign (311).png
dataset/Dataset_BUSI_AN/val/masks/bengin_masks/benign (311).png
dataset/Dataset_BUSI_AN/val/images/bengin_images/benign (11).png
dataset/Dataset_BUSI_AN/val/masks/bengin_masks/benign (11).png
dataset/Dataset_BUSI_AN/val/images/bengin_images/benign (312).png
dataset/Dataset_BUSI_AN/val/

dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (167).png
dataset/Dataset_BUSI_AN/test/masks/bengin_masks/benign (167).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (220).png
dataset/Dataset_BUSI_AN/test/masks/bengin_masks/benign (220).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (104).png
dataset/Dataset_BUSI_AN/test/masks/bengin_masks/benign (104).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (372).png
dataset/Dataset_BUSI_AN/test/masks/bengin_masks/benign (372).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (222).png
dataset/Dataset_BUSI_AN/test/masks/bengin_masks/benign (222).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (150).png
dataset/Dataset_BUSI_AN/test/masks/bengin_masks/benign (150).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (81).png
dataset/Dataset_BUSI_AN/test/masks/bengin_masks/benign (81).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (147).png
dataset/Dat

In [4]:
# from models.shareLayer import create_pair_model
# from models.MLT_net import MTL_classic, MTL_3

# model = MTL_classic(img_size, img_size, depth, nClasses=2)
# model = MTL_3(img_size, img_size, depth, nClasses=2)
# model = create_pair_model(img_size, img_size, depth, nClasses=2)
# model.summary()


# define callbacks
csv_logger = CSVLogger(Name+'.log')

reduce_lr = ReduceLROnPlateau(monitor='classification_output_loss', factor=0.2, patience=20, min_lr=1e-8, mode='auto', verbose=1)

checkpoint_period1 = ModelCheckpoint(Name + '-{epoch:03d}-{classification_output_acc:.4f}.h5',
                                     monitor='classification_output_acc', mode='auto', save_best_only='True')



# define loss and compile the model

opt = Adam(lr=INIT_LR, beta_1=0.9, beta_2=0.99, epsilon=1e-08, decay=0.01)

def binary_crossentropy(y_true, y_pred):
    e=1.0
    return K.mean(-(y_true*K.log(y_pred+K.epsilon())+
                    e*(1-y_true)*K.log(1-y_pred+K.epsilon())),
                  axis=-1)

def tversky(y_true, y_pred, smooth=1):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    alpha = 0.5
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true, y_pred)


from keras.losses import categorical_crossentropy

# model.compile(loss={'segmentation_output': categorical_crossentropy, 'seg-out': tversky_loss, "classification_output": binary_crossentropy},
#               loss_weights={'segmentation_output': 0.25,'seg-out': 0.5, "classification_output": 0.5},
#               optimizer=opt, 
#               metrics={'segmentation_output':'accuracy', 'seg-out':dice_coef, 'classification_output':'accuracy'})

# model.compile(loss={'segmentation_output': categorical_crossentropy, 'seg-out': tversky_loss, "classification_output": binary_crossentropy},
#               loss_weights={'segmentation_output': 0.25,'seg-out': 0.5, "classification_output": 0.5},
#               optimizer=opt, 
#               metrics={'segmentation_output':'accuracy', 'seg-out':dice_coef, 'classification_output':'accuracy'})

# # model.compile(loss={'segmentation_output': tversky_loss, "classification_output": binary_crossentropy},
model.compile(loss={'segmentation_output': tversky_loss, "classification_output": binary_crossentropy},
              loss_weights={'segmentation_output': 0.5, "classification_output": 0.5},
              optimizer=opt,
              metrics={'segmentation_output':['accuracy', dice_coef], 'classification_output':'accuracy'})

# model.compile(loss={'seg_out': 'categorical_crossentropy', 'cls_out': binary_crossentropy},
#               optimizer=Adam(lr=1e-4), metrics={'seg_out':dice_coef, 'cls_out':'accuracy'},
#               loss_weights={'seg_out':0.5, 'cls_out':0.5})


In [ ]:

# print("================================================ begin training ================================================")

# hist = model.fit(train_x,
#                  [train_c_y, train_s_y, train_s_y],
#                  batch_size=batch_size,
#                  epochs=200,
#                  validation_data=(val_x, [val_c_y, val_s_y, val_s_y]),
#                  verbose=1,
#                  callbacks=[reduce_lr, csv_logger],
#                  shuffle=True)

# print("================================================ Saving model ================================================")
# model_filename = Name + ".h5"
# model.save_weights(model_filename)
# print('model saved to:', model_filename)



print("================================================ begin training ================================================")

hist = model.fit(train_x,
                 [train_c_y, train_s_y],
                 batch_size=batch_size,
                 epochs=200,
                 validation_data=(val_x, [val_c_y, val_s_y]),
                 verbose=1,
                 callbacks=[reduce_lr, csv_logger],
                 shuffle=True)

print("================================================ Saving model ================================================")
model_filename = Name + ".h5"
model.save_weights(model_filename)
print('model saved to:', model_filename)




# print("================================================ begin training ================================================")

# hist = model.fit([train_x,train_x],
#                  [train_c_y, train_s_y],
#                  batch_size=batch_size,
#                  epochs=200,
#                  validation_data=([val_x,val_x], [val_c_y, val_s_y]),
#                  verbose=1,
#                  callbacks=[reduce_lr, csv_logger],
#                  shuffle=True)

# print("================================================ Saving model ================================================")
# model_filename = Name + "4.h5"
# model.save_weights(model_filename)
# print('model saved to:', model_filename)


================================================ begin training ================================================
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1000 samples, validate on 64 samples
Epoch 1/200


2022-03-25 02:36:17.383444: I tensorflow/stream_executor/dso_loader.cc:152] successfully opened CUDA library libcublas.so.10.0 locally


1000/1000 [==============================] - 32s 32ms/step - loss: 0.4528 - classification_output_loss: 0.7114 - segmentation_output_loss: 0.1942 - classification_output_acc: 0.5050 - segmentation_output_acc: 0.8680 - segmentation_output_dice_coef: 0.8058 - val_loss: 0.4205 - val_classification_output_loss: 0.6931 - val_segmentation_output_loss: 0.1478 - val_classification_output_acc: 0.6719 - val_segmentation_output_acc: 0.9169 - val_segmentation_output_dice_coef: 0.8522
Epoch 2/200
1000/1000 [==============================] - 25s 25ms/step - loss: 0.4026 - classification_output_loss: 0.6939 - segmentation_output_loss: 0.1113 - classification_output_acc: 0.4960 - segmentation_output_acc: 0.9221 - segmentation_output_dice_coef: 0.8887 - val_loss: 0.3921 - val_classification_output_loss: 0.6931 - val_segmentation_output_loss: 0.0910 - val_classification_output_acc: 0.6719 - val_segmentation_output_acc: 0.9421 - val_segmentation_output_dice_coef: 0.9090
Epoch 3/200
1000/1000 [===========

In [ ]:
# N = np.arange(0, EPOCHS)
# plt.style.use("ggplot")
# plt.figure()
# # plt.plot(N, hist.history["loss"], label="train_loss")
# plt.plot(N, hist.history["classification_output_loss"], label="train_c_loss")
# plt.plot(N, hist.history["segmentation_output_loss"], label="train_s_loss")

# plt.plot(N, hist.history["classification_output_acc"], label="train_c_acc")
# plt.plot(N, hist.history["segmentation_output_acc"], label="train_s_acc")
# plt.plot(N, hist.history["seg-out_dice_coef"], label="train_s_dice")

# # plt.plot(N, hist.history["val_loss"], label="val_loss")
# plt.plot(N, hist.history["val_classification_output_loss"], label="val_c_loss")
# plt.plot(N, hist.history["val_segmentation_output_loss"], label="val_s_loss")

# plt.plot(N, hist.history["val_classification_output_acc"], label="val_c_acc")
# plt.plot(N, hist.history["val_segmentation_output_acc"], label="val_s_acc")
# plt.plot(N, hist.history["val_seg-out_dice_coef"], label="val_s_dice")

# plt.title("Training Loss and Accuracy")
# plt.xlabel("Epoch #")
# plt.ylabel("Loss/Accuracy/DSC")
# plt.legend()
# plt.savefig(os.path.join('output/', Name+'.png'))



N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
# plt.plot(N, hist.history["loss"], label="train_loss")
plt.plot(N, hist.history["classification_output_loss"], label="train_c_loss")
plt.plot(N, hist.history["segmentation_output_loss"], label="train_s_loss")
plt.plot(N, hist.history["classification_output_acc"], label="train_c_acc")
plt.plot(N, hist.history["segmentation_output_acc"], label="train_s_acc")


# plt.plot(N, hist.history["val_loss"], label="val_loss")
plt.plot(N, hist.history["val_classification_output_loss"], label="val_c_loss")
plt.plot(N, hist.history["val_segmentation_output_loss"], label="val_s_loss")
plt.plot(N, hist.history["val_classification_output_acc"], label="val_c_acc")
plt.plot(N, hist.history["val_segmentation_output_acc"], label="val_s_acc")


plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(os.path.join('output/', Name+'.png'))


In [ ]:
print("------Start predicting------")



predictions_c, predictions_s, predictions_s2 = model.predict(test_x, batch_size=32)


print("------------------------------------------------ Classification testing ------------------------------------------------")

print(classification_report(test_c_y.argmax(axis=1), predictions_c.argmax(axis=1), target_names=lb.classes_, digits=4))

print("------------------------------------------------ Segmentation testing ------------------------------------------------")


# evaluate the model
# loss = model.evaluate(test_x, [test_c_y, test_s_y], verbose=0)
loss, cla_loss, seg_loss, seg_loss2, cla_acc, seg_acc, seg_dice_coef = model.evaluate(test_x, [test_c_y, test_s_y, test_s_y], verbose=0)
print('Test total loss:', loss)
print('Test classification loss:', cla_loss)
print('Test segmentation loss:', seg_loss)

print('Test classification accuracy:', cla_acc)
print('Test segmentation accuracy:', seg_acc)
print('Test segmentation dice_coef:', seg_dice_coef)

preds_c, preds_s, preds_s2 = model.predict(test_x, batch_size=8, verbose=1)
test_mask = test_s_y.flatten()
pred_mask = preds_s.flatten()
fpr, tpr, thresholds = roc_curve(test_mask, pred_mask, pos_label=1)
roc_auc = auc(fpr, tpr)
print("AUC:", roc_auc)

# Precision, Recall, accuracy, F1, IoU = p_r_f1_iou(test_s_y, preds_s)
# print('Precision:', Precision)
# print('Recall:', Recall)
# print('m_Iou:', IoU)
# print('F1_score:', F1)



In [ ]:
print("------Start predicting------")



predictions_c, predictions_s, predictions_s2 = model.predict(test_x, batch_size=32)


print("------------------------------------------------ Classification testing ------------------------------------------------")

print(classification_report(test_c_y.argmax(axis=1), predictions_c.argmax(axis=1), target_names=lb.classes_, digits=4))

print("------------------------------------------------ Segmentation testing ------------------------------------------------")


# evaluate the model
# loss = model.evaluate(test_x, [test_c_y, test_s_y], verbose=0)
loss, cla_loss, seg_loss, seg_loss2, cla_acc, seg_acc, seg_dice_coef = model.evaluate(test_x, [test_c_y, test_s_y, test_s_y], verbose=0)
print('Test total loss:', loss)
print('Test classification loss:', cla_loss)
print('Test segmentation loss:', seg_loss)

print('Test classification accuracy:', cla_acc)
print('Test segmentation accuracy:', seg_acc)
print('Test segmentation dice_coef:', seg_dice_coef)

preds_c, preds_s, preds_s2 = model.predict(test_x, batch_size=8, verbose=1)
test_mask = test_s_y.flatten()
pred_mask = preds_s.flatten()
fpr, tpr, thresholds = roc_curve(test_mask, pred_mask, pos_label=1)
roc_auc = auc(fpr, tpr)
print("AUC:", roc_auc)

# Precision, Recall, accuracy, F1, IoU = p_r_f1_iou(test_s_y, preds_s)
# print('Precision:', Precision)
# print('Recall:', Recall)
# print('m_Iou:', IoU)
# print('F1_score:', F1)
